In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import glob
import cv2
import os
import seaborn as sns
import pandas as pd
from skimage.filters import sobel
from skimage.feature import graycomatrix, graycoprops
from skimage.measure import shannon_entropy
from tqdm import tqdm
from ultralytics import YOLO
import torch
import joblib

C:\Users\saiet\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


In [2]:


# === Load YOLO model and setup hook ===
model = YOLO("best_neu.pt")
features = []
def hook(module, input, output):
    features.append(output)
hook_handle = model.model.model[-2].register_forward_hook(hook)  # Hook to last feature layer

# === Class names (update to your own)
CLASS_NAMES = ["crazing", "inclusion","patches","pitted_surface","rolled-in_scale","scratches"]

# === IoU calculator ===
def compute_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = max(1, (boxA[2] - boxA[0]) * (boxA[3] - boxA[1]))
    boxBArea = max(1, (boxB[2] - boxB[0]) * (boxB[3] - boxB[1]))
    return interArea / float(boxAArea + boxBArea - interArea + 1e-6)

# === Parse YOLO TXT annotation ===
def parse_yolo_txt_annotation(txt_path, img_width, img_height):
    gt_boxes = []
    gt_labels = []
    if not os.path.exists(txt_path):
        return gt_boxes, gt_labels
    with open(txt_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            class_id = int(parts[0])
            x_center = float(parts[1]) * img_width
            y_center = float(parts[2]) * img_height
            w = float(parts[3]) * img_width
            h = float(parts[4]) * img_height
            x1 = int(x_center - w / 2)
            y1 = int(y_center - h / 2)
            x2 = int(x_center + w / 2)
            y2 = int(y_center + h / 2)
            gt_boxes.append([x1, y1, x2, y2])
            gt_labels.append(class_id)
    return gt_boxes, gt_labels

# === GLCM feature extractor ===
def extract_glcm_features(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #img = np.array(img)

    image_dataset = pd.DataFrame()

    

    df = pd.DataFrame()
    GLCM = graycomatrix(img, [1], [0])       
    GLCM_Energy = graycoprops(GLCM, 'energy')[0]
    df['Energy'] = GLCM_Energy
    GLCM_corr = graycoprops(GLCM, 'correlation')[0]
    df['Corr'] = GLCM_corr       
    GLCM_diss = graycoprops(GLCM, 'dissimilarity')[0]
    df['Diss_sim'] = GLCM_diss       
    GLCM_hom = graycoprops(GLCM, 'homogeneity')[0]
    df['Homogen'] = GLCM_hom       
    GLCM_contr = graycoprops(GLCM, 'contrast')[0]
    df['Contrast'] = GLCM_contr


    GLCM2 = graycomatrix(img, [3], [0])       
    GLCM_Energy2 = graycoprops(GLCM2, 'energy')[0]
    df['Energy2'] = GLCM_Energy2
    GLCM_corr2 = graycoprops(GLCM2, 'correlation')[0]
    df['Corr2'] = GLCM_corr2       
    GLCM_diss2 = graycoprops(GLCM2, 'dissimilarity')[0]
    df['Diss_sim2'] = GLCM_diss2       
    GLCM_hom2 = graycoprops(GLCM2, 'homogeneity')[0]
    df['Homogen2'] = GLCM_hom2       
    GLCM_contr2 = graycoprops(GLCM2, 'contrast')[0]
    df['Contrast2'] = GLCM_contr2

    GLCM3 = graycomatrix(img, [5], [0])       
    GLCM_Energy3 = graycoprops(GLCM3, 'energy')[0]
    df['Energy3'] = GLCM_Energy3
    GLCM_corr3 = graycoprops(GLCM3, 'correlation')[0]
    df['Corr3'] = GLCM_corr3       
    GLCM_diss3 = graycoprops(GLCM3, 'dissimilarity')[0]
    df['Diss_sim3'] = GLCM_diss3       
    GLCM_hom3 = graycoprops(GLCM3, 'homogeneity')[0]
    df['Homogen3'] = GLCM_hom3       
    GLCM_contr3 = graycoprops(GLCM3, 'contrast')[0]
    df['Contrast3'] = GLCM_contr3

    GLCM4 = graycomatrix(img, [0], [np.pi/4])       
    GLCM_Energy4 = graycoprops(GLCM4, 'energy')[0]
    df['Energy4'] = GLCM_Energy4
    GLCM_corr4 = graycoprops(GLCM4, 'correlation')[0]
    df['Corr4'] = GLCM_corr4       
    GLCM_diss4 = graycoprops(GLCM4, 'dissimilarity')[0]
    df['Diss_sim4'] = GLCM_diss4       
    GLCM_hom4 = graycoprops(GLCM4, 'homogeneity')[0]
    df['Homogen4'] = GLCM_hom4       
    GLCM_contr4 = graycoprops(GLCM4, 'contrast')[0]
    df['Contrast4'] = GLCM_contr4
    
    GLCM5 = graycomatrix(img, [0], [np.pi/2])       
    GLCM_Energy5 = graycoprops(GLCM5, 'energy')[0]
    df['Energy5'] = GLCM_Energy5
    GLCM_corr5 = graycoprops(GLCM5, 'correlation')[0]
    df['Corr5'] = GLCM_corr5       
    GLCM_diss5 = graycoprops(GLCM5, 'dissimilarity')[0]
    df['Diss_sim5'] = GLCM_diss5       
    GLCM_hom5 = graycoprops(GLCM5, 'homogeneity')[0]
    df['Homogen5'] = GLCM_hom5       
    GLCM_contr5 = graycoprops(GLCM5, 'contrast')[0]
    df['Contrast5'] = GLCM_contr5

    df =df.drop(["Corr4","Diss_sim4","Contrast4","Corr5","Diss_sim5","Homogen3","Homogen4","Homogen5","Contrast5","Energy5"],axis=1)
    return df.values.flatten()


# === YOLO feature extractor ===

def extract_yolo_features(image):
    im = cv2.resize(image, (640, 640))
    im = im.astype(np.float32) / 255.0
    im = torch.from_numpy(im.transpose(2, 0, 1)).unsqueeze(0)

    # ✅ Match device with model (GPU or CPU)
    device = next(model.model.parameters()).device
    im = im.to(device)

    features.clear()
    with torch.no_grad():
        _ = model.model(im)

    feat_tensor = features[0].cpu()  # optionally move to CPU before converting to numpy
    feat_vector = torch.nn.functional.adaptive_avg_pool2d(feat_tensor, 1).view(feat_tensor.shape[0], -1)
    return feat_vector.squeeze().numpy()


# === Thermal conversion ===
def convert_to_thermal(img_gray):
    return cv2.applyColorMap(img_gray, cv2.COLORMAP_INFERNO)

# === Main pipeline ===
def process_images(image_folder, annotation_folder):
    all_data = []
    all_targets = []

    for file in tqdm(os.listdir(image_folder), desc="Processing"):
        if not file.lower().endswith((".jpg", ".jpeg", ".png")):
            continue

        filename = os.path.splitext(file)[0]
        img_path = os.path.join(image_folder, file)
        annot_path = os.path.join(annotation_folder, filename + ".txt")

        original = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if original is None:
            continue

        h, w = original.shape
        thermal = convert_to_thermal(original)
        original_color = cv2.cvtColor(original, cv2.COLOR_GRAY2BGR)

        gt_boxes, gt_labels = parse_yolo_txt_annotation(annot_path, w, h)

        results = model(original_color, conf=0.05, iou=0.0, agnostic_nms=True)
        det_boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)

        for x1, y1, x2, y2 in det_boxes:
            norm_crop = original_color[y1:y2, x1:x2]
            thermal_crop = thermal[y1:y2, x1:x2]
            if norm_crop.size == 0 or thermal_crop.size == 0:
                continue

            try:
                yolo_vec = extract_yolo_features(norm_crop)
                glcm_vec = extract_glcm_features(thermal_crop)
                row = list(yolo_vec) + list(glcm_vec)
                all_data.append(row)

                # Match with closest GT box
                best_label = "unknown"
                max_iou = 0
                for gt_box, label in zip(gt_boxes, gt_labels):
                    img_h, img_w = original_color.shape[:2]
                    x_center = ((x1 + x2) / 2) / img_w
                    y_center = ((y1 + y2) / 2) / img_h
                    width = (x2 - x1) / img_w
                    height = (y2 - y1) / img_h

                    updated_annotation = f"{label} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n"

                    # Prepare output directory
                    os.makedirs("predicted_annotations", exist_ok=True)
                    new_annotation_file = os.path.join("predicted_annotations", os.path.splitext(filename)[0] + ".txt")
                    annotation_filename = os.path.splitext(filename)[0] + ".txt"

                    output_path = os.path.join("predicted_annotations", annotation_filename)
                    with open(output_path, "a") as f:
                        f.write(updated_annotation)


                    iou = compute_iou([x1, y1, x2, y2], gt_box)
                    if iou > max_iou:
                        max_iou = iou
                        best_label = label
                all_targets.append(best_label)

            except Exception as e:
                print(f"⚠️ Error in {file}: {e}")

    # Build DataFrame
    yolo_cols = [f"{i}" for i in range(len(yolo_vec))]
    # Column names for GLCM features
    glcm_cols = ['Energy', 'Corr', 'Diss_sim', 'Homogen', 'Contrast',
             'Energy2', 'Corr2', 'Diss_sim2', 'Homogen2', 'Contrast2',
             'Energy3', 'Corr3', 'Diss_sim3', 'Contrast3', 'Energy4']
    df = pd.DataFrame(all_data, columns=yolo_cols + glcm_cols)

    

    # 2. Combine feature vectors into a row
    #row = list(yolo_vec) + list(glcm_vec)

# 3. When all rows are ready, construct DataFrame
    df = pd.DataFrame(all_data, columns=yolo_cols + glcm_cols)
    #df = pd.DataFrame(all_data)
    print("✅ Final feature matrix shape:", df.shape)
    return df,all_targets

# === Run pipeline ===
df_combined,targets = process_images("Data/NEU-DET/NEU-DET/IMAGES","VOC_lab")



##############################################
best_neu.pt


Processing:   0%|                                                                             | 0/1800 [00:00<?, ?it/s]


0: 640x640 2 crazings, 19.2ms
Speed: 7.8ms preprocess, 19.2ms inference, 185.3ms postprocess per image at shape (1, 3, 640, 640)


Processing:   0%|                                                                   | 1/1800 [00:03<1:46:15,  3.54s/it]


0: 640x640 3 crazings, 19.2ms
Speed: 4.9ms preprocess, 19.2ms inference, 3.3ms postprocess per image at shape (1, 3, 640, 640)


...


0: 640x640 2 inculsions, 2 scratchess, 19.7ms
Speed: 5.8ms preprocess, 19.7ms inference, 3.7ms postprocess per image at shape (1, 3, 640, 640)


Processing: 100%|█████████████████████████████████████████████████████████████████▉| 1798/1800 [07:07<00:00,  2.80it/s]


0: 640x640 1 scratches, 18.7ms
Speed: 5.3ms preprocess, 18.7ms inference, 3.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 scratches, 18.0ms
Speed: 4.7ms preprocess, 18.0ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)


Processing: 100%|██████████████████████████████████████████████████████████████████| 1800/1800 [07:07<00:00,  4.21it/s]


✅ Final feature matrix shape: (5062, 271)


In [3]:
df_combined

,0,1,2,3,4,5,6,7,8,9,...,Energy2,Corr2,Diss_sim2,Homogen2,Contrast2,Energy3,Corr3,Diss_sim3,Contrast3,Energy4
0,-0.000801,0.179746,-0.044113,0.149404,-0.216330,0.657885,0.327573,0.233934,0.256287,-0.087819,...,0.016388,0.397753,22.311317,0.046469,819.716742,0.016134,0.251035,24.997656,1016.545448,0.115271
1,0.188967,0.278953,-0.087048,0.220420,0.291997,0.870558,0.212587,0.557211,0.180714,0.173702,...,0.018626,0.448976,20.106406,0.053793,669.683784,0.018374,0.349351,21.731127,783.602004,0.121006
2,0.192903,0.265468,-0.059010,0.404200,0.235915,0.811797,0.375189,0.579805,0.184275,0.164654,...,0.019092,0.753674,16.514645,0.064906,476.698907,0.018660,0.693034,18.265635,592.771271,0.110366
3,0.047972,0.220046,-0.002095,0.275697,0.314334,0.694794,0.243044,0.202302,0.312630,0.205469,...,0.022977,0.673694,12.882275,0.078550,274.389051,0.022503,0.597872,14.056609,329.252438,0.127987
4,0.158983,0.253079,-0.055906,0.171083,0.332521,0.923105,0.197323,0.542001,0.228141,0.260887,...,0.018811,0.591063,17.835833,0.060105,539.322292,0.018301,0.511138,19.713682,644.604941,0.117771
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5057,0.041215,0.286292,0.079802,-0.153371,0.090923,0.244393,0.179788,0.071105,0.173398,-0.049650,...,0.029379,0.505342,23.421368,0.082767,1182.274359,0.028391,0.122907,34.859596,2171.983838,0.133294
5058,-0.078939,0.329500,0.090674,-0.160737,0.213955,0.161361,0.270035,-0.012299,0.209257,0.051388,...,0.032800,0.320034,22.206169,0.071272,1033.092532,0.032506,0.052306,29.527922,1551.035714,0.141694
5059,0.015294,0.067070,0.156074,0.181939,0.327714,0.152363,0.015314,0.153501,0.244622,0.143916,...,0.113636,-0.048280,15.306818,0.097663,377.625000,0.122663,-0.445182,19.055556,512.694444,0.242226
5060,0.220264,0.052139,0.006646,0.188444,0.640382,0.829487,0.152037,0.584978,0.095894,0.307587,...,0.040919,0.953853,4.507772,0.237536,43.961375,0.039951,0.942762,5.076154,55.301761,0.135716


In [4]:
len(targets)

5062

In [5]:
targets

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 'unknown',
 0,
 0,
 0,
 'unknown',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 'unknown',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 'unknown',
 0,
 'unknown',
 0,
 'unknown',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 'unknown',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 'unknown',
 'unknown',
 'unknown',
 0,
 'unknown',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 'unknown',
 0,
 0,
 0,
 0,
 'unknown',
 'unknown',
 0,
 0,
 0,
 0,
 0,
 0,
 'unknown',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 'unknown',
 0,
 'unknown',
 0,
 0,
 0,
 0,
 0,
 'unknown',
 0,
 'unknown',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 'unknown',
 'unknown',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 'unknown',
 'unknown',
 0,
 0,
 0,
 0,
 0,
 0,
 'unknown',
 0,
 'unknown',
 'unknown',
 0,
 'unknown',
 0,
 0,
 0,
 'unknown',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 'unknown',
 0,
 'unknown',
 0,
 'unknown',
 0,
 0,
 0,
 0,
 'unknown',
 0,
 0,
 0,
 0,


In [6]:
scaler = joblib.load("scaler.pkl")
pca = joblib.load("pca.pkl")
svm_model = joblib.load("svm_model.pkl")
le = joblib.load("label_encoder.pkl")


In [7]:
X_new = df_combined.drop(columns=["target"], errors='ignore')

# Scale and reduce dimensionality
X_scaled = scaler.transform(X_new)
X_pca = pca.transform(X_scaled)
#print(X_pca.head())
# Predict using the trained SVM model
predicted_classes = svm_model.predict(X_pca)

# Add to the dataframe
df_combined["svm_pred"] = predicted_classes
df_combined["svm_pred_label"] = le.inverse_transform(df_combined["svm_pred"])

print("✅ Predictions added to dataframe!")
print(df_combined[["svm_pred"]].head())



✅ Predictions added to dataframe!
   svm_pred
0         5
1         5
2         5
3         5
4         5


In [8]:
df_combined

,0,1,2,3,4,5,6,7,8,9,...,Diss_sim2,Homogen2,Contrast2,Energy3,Corr3,Diss_sim3,Contrast3,Energy4,svm_pred,svm_pred_label
0,-0.000801,0.179746,-0.044113,0.149404,-0.216330,0.657885,0.327573,0.233934,0.256287,-0.087819,...,22.311317,0.046469,819.716742,0.016134,0.251035,24.997656,1016.545448,0.115271,5,scratches
1,0.188967,0.278953,-0.087048,0.220420,0.291997,0.870558,0.212587,0.557211,0.180714,0.173702,...,20.106406,0.053793,669.683784,0.018374,0.349351,21.731127,783.602004,0.121006,5,scratches
2,0.192903,0.265468,-0.059010,0.404200,0.235915,0.811797,0.375189,0.579805,0.184275,0.164654,...,16.514645,0.064906,476.698907,0.018660,0.693034,18.265635,592.771271,0.110366,5,scratches
3,0.047972,0.220046,-0.002095,0.275697,0.314334,0.694794,0.243044,0.202302,0.312630,0.205469,...,12.882275,0.078550,274.389051,0.022503,0.597872,14.056609,329.252438,0.127987,5,scratches
4,0.158983,0.253079,-0.055906,0.171083,0.332521,0.923105,0.197323,0.542001,0.228141,0.260887,...,17.835833,0.060105,539.322292,0.018301,0.511138,19.713682,644.604941,0.117771,5,scratches
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5057,0.041215,0.286292,0.079802,-0.153371,0.090923,0.244393,0.179788,0.071105,0.173398,-0.049650,...,23.421368,0.082767,1182.274359,0.028391,0.122907,34.859596,2171.983838,0.133294,5,scratches
5058,-0.078939,0.329500,0.090674,-0.160737,0.213955,0.161361,0.270035,-0.012299,0.209257,0.051388,...,22.206169,0.071272,1033.092532,0.032506,0.052306,29.527922,1551.035714,0.141694,5,scratches
5059,0.015294,0.067070,0.156074,0.181939,0.327714,0.152363,0.015314,0.153501,0.244622,0.143916,...,15.306818,0.097663,377.625000,0.122663,-0.445182,19.055556,512.694444,0.242226,5,scratches
5060,0.220264,0.052139,0.006646,0.188444,0.640382,0.829487,0.152037,0.584978,0.095894,0.307587,...,4.507772,0.237536,43.961375,0.039951,0.942762,5.076154,55.301761,0.135716,5,scratches


In [9]:
df_combined["svm_pred_label"].unique()


array(['scratches'], dtype=object)